# Lab 1 Part II: UCI-HAR 1D CNN

## Imports

In [2]:
from pathlib import Path
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, MaxPool1D, Flatten, Dense, Activation
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical

## Load and format UCI-HAR dataset (raw data)

In [3]:
dataset_path = get_file(None, "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip", extract=True, file_hash="53e099237392e0b9602f8c38f578bd8f") # Download, cache and extract UCI-HAR
dataset_dir = Path(dataset_path).parent

def load_set(dataset_dir, part: str): # Load separate sensor signals and combine them into a single array, load labels separately
    data = np.hstack([np.loadtxt(dataset_dir/'UCI HAR Dataset'/part/'Inertial Signals'/f'{sensor}_{axis}_{part}.txt')
                for sensor in ('body_acc', 'body_gyro', 'total_acc')
                    for axis in ('x', 'y', 'z')]).reshape((-1, 128, 9))
    labels = to_categorical(np.loadtxt(dataset_dir/'UCI HAR Dataset'/part/f'y_{part}.txt') - 1)
    return data, labels

x_train, y_train = load_set(dataset_dir, 'train')
x_test, y_test = load_set(dataset_dir, 'test')

## Export complete test dataset (2947 vectors)

In [4]:
np.savetxt('x_test_uci-har.csv', x_test.reshape((x_test.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_uci-har.csv', y_test, delimiter=',', fmt='%s')

## Export small test dataset (250 vectors)

In [5]:
x_test_250 = x_test[0:250]
y_test_250 = y_test[0:250]
np.savetxt('x_test_uci-har_250.csv', x_test_250.reshape((x_test_250.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_uci-har_250.csv', y_test_250, delimiter=',', fmt='%s')

## Build model

In [35]:
# model = Sequential()
# model.add(Input(shape=(128, 9)))
# model.add(Conv1D(filters=2, kernel_size=3, activation='relu'))
# model.add(Flatten())
# model.add(Dense(units=6))
# model.add(Activation('softmax')) # SoftMax activation needs to be separate from Dense to remove it later on
# # EXPLORE Learning Rate
# opt = tf.keras.optimizers.Adam(learning_rate=10e-3)
# model.summary()
# model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])


model = Sequential()
model.add(Input(shape=(128, 9))) # 128 data points form sensors | 9 time steps
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=2))
# model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(units=6))
model.add(Activation('softmax')) # SoftMax activation needs to be separate from Dense to remove it later on

# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(lr=5e-3)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_9 (Conv1D)           (None, 126, 32)           896       
                                                                 
 conv1d_10 (Conv1D)          (None, 124, 64)           6208      
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 62, 64)           0         
 1D)                                                             
                                                                 
 flatten_5 (Flatten)         (None, 3968)              0         
                                                                 
 dense_5 (Dense)             (None, 6)                 23814     
                                                                 
 activation_5 (Activation)   (None, 6)                 0         
                                                      

## Train model

In [36]:
hist1 = model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test))
hist2 = model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test))
hist3 = model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test))

Epoch 1/3
  5/230 [..............................] - ETA: 2s - loss: 1.4431 - categorical_accuracy: 0.4188  

2023-03-11 17:55:43.527034: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


229/230 [============================>.] - ETA: 0s - loss: 0.2698 - categorical_accuracy: 0.8919

2023-03-11 17:55:46.270919: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


230/230 [==============================] - 4s 15ms/step - loss: 0.2693 - categorical_accuracy: 0.8921 - val_loss: 0.2179 - val_categorical_accuracy: 0.9084
Epoch 2/3
230/230 [==============================] - 3s 14ms/step - loss: 0.1284 - categorical_accuracy: 0.9495 - val_loss: 0.3376 - val_categorical_accuracy: 0.8911
Epoch 3/3
230/230 [==============================] - 3s 14ms/step - loss: 0.0917 - categorical_accuracy: 0.9619 - val_loss: 0.2970 - val_categorical_accuracy: 0.9104
Epoch 1/3
230/230 [==============================] - 3s 14ms/step - loss: 0.0845 - categorical_accuracy: 0.9671 - val_loss: 0.3110 - val_categorical_accuracy: 0.9206
Epoch 2/3
230/230 [==============================] - 3s 14ms/step - loss: 0.0699 - categorical_accuracy: 0.9687 - val_loss: 0.2712 - val_categorical_accuracy: 0.9332
Epoch 3/3
230/230 [==============================] - 3s 14ms/step - loss: 0.0536 - categorical_accuracy: 0.9774 - val_loss: 0.3427 - val_categorical_accuracy: 0.9332
Epoch 1/3
230/

## Evaluate model on complete test dataset

In [17]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

93/93 - 1s - loss: 0.3261 - categorical_accuracy: 0.9223 - 741ms/epoch - 8ms/step
67/93 [====================>.........] - ETA: 0s

2023-03-11 17:20:22.077618: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


93/93 [==============================] - 0s 3ms/step
tf.Tensor(
[[489   5   1   0   1   0]
 [ 20 449   2   0   0   0]
 [ 44  34 342   0   0   0]
 [  1   3   0 380 107   0]
 [  1   1   0   9 521   0]
 [  0   0   0   0   0 537]], shape=(6, 6), dtype=int32)


## Evaluate model on small test dataset

In [18]:
model.evaluate(x_test_250, y_test_250, verbose=2)
pred_test_250 = model.predict(x_test_250)
print(tf.math.confusion_matrix(y_test_250.argmax(axis=1), pred_test_250.argmax(axis=1)))

8/8 - 0s - loss: 0.6598 - categorical_accuracy: 0.8000 - 91ms/epoch - 11ms/step
8/8 [==============================] - 0s 5ms/step
tf.Tensor(
[[53  0  0  0  0  0]
 [ 0 25  0  0  0  0]
 [ 1 16  7  0  0  0]
 [ 0  0  0 13 33  0]
 [ 0  0  0  0 54  0]
 [ 0  0  0  0  0 48]], shape=(6, 6), dtype=int32)


## Save trained model

In [10]:
model.save('lab1_part2_uci-har.h5')